## Large Language Models, the Chat Format and Tokens
##### Setup
Load the API key and relevant Python libaries.

In [17]:
#!pip install openai
#!pip install tiktoken
#!pip install dotenv

In [1]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]

**Helper function**

In [12]:
client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])


def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    # response = openai.ChatCompletion.create(
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

##### Prompt the model and get completion

In [13]:
response = get_completion("What is the capital of France?")
print(response)

The capital of France is Paris.


##### Tokens

In [14]:
response = get_completion("Take the letters in lollipop and reverse them")
print(response)

pilpolol


In [15]:
# that was incorrect. What about this?
response = get_completion(
    """Take the letters in \
l-o-l-l-i-p-o-p and reverse them"""
)
print(response)

p-o-p-i-l-l-o-l


##### Helper function (chat format)¶
Here's the helper function we'll use in the following cells

In [16]:
def get_completion_from_messages(
    messages,
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=500,
):
    response = openai.chat.completions.create(  # ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
        max_tokens=max_tokens,  # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message.content  # ["content"]

In [17]:
messages = [
    {
        "role": "system",
        "content": """You are an assistant who responds in the style of Dr Seuss.""",
    },
    {
        "role": "user",
        "content": """write me a very short poem about a happy carrot""",
    },
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh happy carrot, so joyful and bright,
Grown in the garden, a pure delight.
With a leafy green top and orange so cheery,
You make us all smile, oh so clearly!
Crunchy and sweet, you make a tasty snack,
Oh happy carrot, we'll never look back.


In [18]:
# length
messages = [
    {"role": "system", "content": "All your responses must be one sentence long."},
    {"role": "user", "content": "write me a story about a happy carrot"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Once there was a cheerful carrot named Oliver who made everyone smile with his bright orange glow.


In [19]:
# combined
messages = [
    {
        "role": "system",
        "content": """You are an assistant who responds in the style of Dr Seuss. \
All your responses must be one sentence long.""",
    },
    {"role": "user", "content": """write me a story about a happy carrot"""},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In a garden so bright, was a carrot so right, always grinning with all its might.


In [30]:
def get_completion_and_token_count(
    messages,
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=500,
):
    response = openai.chat.completions.create(  # ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message.content  # ["content"]

    # token_dict = {
    #     "prompt_tokens": response["usage"]["prompt_tokens"],
    #     "completion_tokens": response["usage"]["completion_tokens"],
    #     "total_tokens": response["usage"]["total_tokens"],
    # }
    token_dict = {
        "completion_tokens": response.usage.completion_tokens,
        "prompt_tokens": response.usage.prompt_tokens,
        "total_tokens": response.usage.total_tokens,
    }

    return response, content, token_dict

In [31]:
messages = [
    {
        "role": "system",
        "content": """You are an assistant who responds in the style of Dr Seuss.""",
    },
    {"role": "user", "content": """write me a very short poem  about a happy carrot"""},
]
# response, token_dict, content = get_completion_and_token_count(messages)
response, content, token_dict = get_completion_and_token_count(messages)
print(content)
print(token_dict)

Oh, the happy carrot, so bright and so orange,
In the garden it grows, a joyful storage.
With a leafy green top and a crunchy bite,
It brings smiles to all, such a delightful sight!
{'completion_tokens': 44, 'prompt_tokens': 36, 'total_tokens': 80}


In [32]:
print(token_dict)

{'completion_tokens': 44, 'prompt_tokens': 36, 'total_tokens': 80}


In [33]:
print(response)  # the entire original response from LLM

ChatCompletion(id='chatcmpl-9gxoUNmA7MLeSdwpBje65Yf7nKHiu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Oh, the happy carrot, so bright and so orange,\nIn the garden it grows, a joyful storage.\nWith a leafy green top and a crunchy bite,\nIt brings smiles to all, such a delightful sight!', role='assistant', function_call=None, tool_calls=None))], created=1720026726, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=44, prompt_tokens=36, total_tokens=80))
